In [1]:
import spacy

In [2]:
from tqdm import tqdm
import spacy
import random
import json
import re

In [3]:
training_data = []
lines=[]
with open('Name_trainer.json', 'r',encoding="utf8") as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    entities = []
    if(data['annotation'] is not None):
        for annotation in data['annotation']:
              if(annotation is not None):
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))

    if(entities != []):
        training_data.append((text, {"entities" : entities}))

In [4]:
len(training_data)

123

In [5]:
training_data[4:15]

[('Hi Miss Kumari Sneha, Your age is 23 years.So you stated the problems of loss of appetite,poor appetite.Report shows that you are suffering from breast cancer.The prescribed medicine you have to take are ethosuximide 500 mg once a day for 5 days,amoxicillin Solid ml thrice a day for 5 days.You should avoid sugary snacks,You should focus on lifestyle changes.',
  {'entities': [(2, 20, 'Name')]}),
 ('You are Vikash maharaja of age 35 years.So, you have the problems of Unusual vaginal bleeding,discharge.Report shows that you are suffering from AIDS.You have to take the medicines dexamethasone Oral liquid, amitriptyline Tablet: 75 mg,enicillamine Solid oral: 500 mg, 1 g (as sodium salt),ceftazidime Powder for injection: 250 mg or 1 g (as pentahydrate) in vial.You should not get too much stressed,\nYou should do water splashes on your face.',
  {'entities': [(8, 23, 'Name')]}),
 ('Hi Mr Kamlesh Hansda of age 23 years.The symptoms you are having are Slow-healing wounds,Nausea.You are suff

In [6]:
## Training a blank spacy english model.

nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 123/123 [00:00<00:00, 194239.23it/s]


Statring iteration 0
{'ner': 476.3626137987065}
Statring iteration 1
{'ner': 174.42816567200663}
Statring iteration 2
{'ner': 202.43390492135129}
Statring iteration 3
{'ner': 581.7451830502296}
Statring iteration 4
{'ner': 339.53580060551707}
Statring iteration 5
{'ner': 640.8368433617105}
Statring iteration 6
{'ner': 386.3587355098326}
Statring iteration 7
{'ner': 504.3176568816565}
Statring iteration 8
{'ner': 568.0114353846583}
Statring iteration 9
{'ner': 448.06543489909154}
Statring iteration 10
{'ner': 495.26749445407455}
Statring iteration 11
{'ner': 428.89080460056135}
Statring iteration 12
{'ner': 419.24027060907616}
Statring iteration 13
{'ner': 551.7962856395391}
Statring iteration 14
{'ner': 233.26353091017245}
Statring iteration 15
{'ner': 439.1780735861503}
Statring iteration 16
{'ner': 191.32717677221888}
Statring iteration 17
{'ner': 6.382811917109723}
Statring iteration 18
{'ner': 3.744885831555496}
Statring iteration 19
{'ner': 1.9379337076218286}


In [8]:
nlp.to_disk("new_model_5")

In [9]:
import spacy
nlp4=spacy.load("new_model_5")

In [10]:
doc=nlp4(open("./MEDICAL_REPORTS/TEST_REPORT_11.txt").read())

In [11]:
doc

hello,Piyush Jagga, You are 25 years old.According to your problems i have read you are having the problems of Persistent lumps,swollen glands.You are suffering from problem of disease HIV.Medicines diazepam Gel or rectal solution: 5 mg/ mL in 0.5 mL 2‐ mL 4‐mL tubes,lorazepam Parenteral formulation: 2 mg/ mL in 1‐ mL ampoule; 4 mg/ mL in 1‐ mL ampoule.avoid lifting heavy weights.You should maintain your Body Mass Index.I would suggest doing more of effort.

In [12]:
for entity in doc.ents:
    print(entity.text)

Piyush Jagga
